# Import Library

In [2]:
!pip install tqdm

You should consider upgrading via the 'C:\Users\yuins\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
import datetime
import os
import time
import zipfile
from tqdm import tqdm

import requests
# import yaml

# Set path

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/2023_AI/data/"

In [4]:
%cd /content/drive/MyDrive/2023_AI/

/content/drive/MyDrive/2023_AI


In [5]:
%pwd

'/content/drive/MyDrive/2023_AI'

# DataSet Download
Referred https://github.com/saif-mahmud/self-attention-HAR/blob/main/configs/data.yaml

In [14]:
# basic dictionary of data download
config = {'data_dir': {'raw' : 'data/raw', 'processed' : 'data\processed'},
          'pamap2' : {'source' : 'https://archive.ics.uci.edu/ml/machine-learning-databases/00231/PAMAP2_Dataset.zip',
                      'destination' : 'pamap2.zip',
                      'train_files' : [ 'subject101.dat', 'subject102.dat', 'subject103.dat','subject104.dat', 'subject107.dat', 'subject108.dat', 'subject109.dat' ],
                      'validation_files' : [ 'subject105.dat' ],
                      'test_files' : [ 'subject106.dat' ],
                      'output_file': 'pamap2_106.h5',
                      'feature_columns' : [ 1, 4, 5, 6, 10, 11, 12, 21, 22, 23, 27, 28, 29, 38, 39, 40, 44, 45, 46 ],
                      'window_size' : 33
                      }}
           
#   raw: data/raw
#   processed: data/processed

# pamap2:
#   source: https://archive.ics.uci.edu/ml/machine-learning-databases/00231/PAMAP2_Dataset.zip
#   destination: pamap2.zip
#   train_files: [ 'subject101.dat', 'subject102.dat', 'subject103.dat','subject104.dat', 'subject107.dat', 'subject108.dat', 'subject109.dat' ]
#   validation_files: [ 'subject105.dat' ]
#   test_files: [ 'subject106.dat' ]
#   output_file: 'pamap2_106.h5'
#   feature_columns: [ 1, 4, 5, 6, 10, 11, 12, 21, 22, 23, 27, 28, 29, 38, 39, 40, 44, 45, 46 ]
#   window_size: 33

In [6]:
import argparse
import datetime
import os
import time
import zipfile
from tqdm import tqdm

import requests
# import yaml


def get_dataset(url: str, data_directory: str, file_name: str, unzip: bool):
    if not os.path.exists('data/'):
        os.mkdir('data/')

    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

    if not os.path.exists(data_directory):
        os.makedirs(data_directory)

    if not os.path.exists(os.path.join(data_directory, file_name)):
        print(f'GETTING DATASET [{file_name}] ...')

        response = requests.get(url, stream=True)
        data_file = open(os.path.join(data_directory, file_name), 'wb')

        total_size = int(response.headers.get("Content-Length", 0))
        chunk_size = 1024

        for chunk in tqdm(iterable=response.iter_content(chunk_size=chunk_size), total=total_size / chunk_size,
                          unit='KB', unit_scale=True, unit_divisor=chunk_size):
            data_file.write(chunk)

        data_file.close()

        if unzip:
            print(f'Unzipping [{file_name}] ...')
            with zipfile.ZipFile(os.path.join(data_directory, file_name), 'r') as zip_ref:
                zip_ref.extractall(os.path.join(data_directory, file_name.split('.')[0]))
                os.remove(os.path.join(data_directory, file_name))

        print('\n---DATASET DOWNLOAD COMPLETE---')

    else:
        print(f'Requested dataset exists in {data_directory}')

In [7]:
get_dataset(url=config['pamap2']['source'], data_directory=config['data_dir']['raw'],
                file_name=config['pamap2']['destination'], unzip=True)

2023-05-27 19:33:36
GETTING DATASET [pamap2.zip] ...


100%|█████████▉| 656k/656k [01:46<00:00, 6.72kKB/s] C:\Users\yuins\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\std.py:524: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 656k/656k [01:46<00:00, 6.29kKB/s]


Unzipping [pamap2.zip] ...


PermissionError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'data/raw\\pamap2.zip'

# Reading Data

In [12]:
import csv
import os

import h5py #  매우 크고 복잡한 대용량 데이터를 저장하기 위한 파일 형식
import matplotlib.pyplot as plt
import numpy as np

plt.style.use('ggplot')


class data_reader: 
    def __init__(self, train_test_files, use_columns, output_file_name):
        if not os.path.exists(output_file_name): # output file name
            self.data, self.idToLabel = self.readPamap2(train_test_files, use_columns)
            self.save_data(output_file_name)

    def save_data(self, output_file_name):
        print(output_file_name)
        f = h5py.File(output_file_name, 'w')
        for key in self.data:
            f.create_group(key)
            for field in self.data[key]:
                f[key].create_dataset(field, data=self.data[key][field])
        f.close()

    @property #getter
    def train(self):
        return self.data['train']

    @property #getter
    def test(self):
        return self.data['test']

    def readPamap2(self, train_test_files, use_columns):
        files = train_test_files
        label_map = [
            # (0, 'other'),
            (1, 'lying'),
            (2, 'sitting'),
            (3, 'standing'),
            (4, 'walking'),
            (5, 'running'),
            (6, 'cycling'),
            (7, 'Nordic walking'),
            (9, 'watching TV'),
            (10, 'computer work'),
            (11, 'car driving'),
            (12, 'ascending stairs'),
            (13, 'descending stairs'),
            (16, 'vacuum cleaning'),
            (17, 'ironing'),
            (18, 'folding laundry'),
            (19, 'house cleaning'),
            (20, 'playing soccer'),
            (24, 'rope jumping')
        ] # column과 그것에 해당하는 이름 작성
        labelToId = {str(x[0]): i for i, x in enumerate(label_map)} # 위의 것을 "1(column label) : 0(부터 시작하는 인덱스 순서)로 바꾸는 딕셔너리 정의"
        idToLabel = [x[1] for x in label_map] # Label들
        cols = use_columns
        data = {dataset: self.readPamap2Files(files[dataset], cols, labelToId)
                for dataset in ('train', 'test', 'validation')}
        return data, idToLabel

    def readPamap2Files(self, filelist, cols, labelToId):
        data = []
        labels = []
        for i, filename in enumerate(filelist):
            # print('Reading file %d of %d' % (i + 1, len(filelist)))
            with open('data/raw/pamap2/PAMAP2_Dataset/Protocol/%s' % filename, 'r') as f:
                reader = csv.reader(f, delimiter=' ') # 읽어옴
                for line in reader:
                    elem = []
                    # not including the non related activity
                    if line[1] == "0": # others인 경우 넘어가기
                        continue
                    for ind in cols: 
                        elem.append(line[ind])
                    if sum([x == 'NaN' for x in elem]) < 9:
                        data.append([float(x) / 1000 for x in elem[:-1]]) # data.append
                        labels.append(labelToId[elem[0]])

        return {'inputs': np.asarray(data), 'targets': np.asarray(labels, dtype=int) + 1}


def read_dataset(train_test_files, use_columns, output_file_name):
    print('[Reading PAMAP2] ...')
    data_reader(train_test_files, use_columns, output_file_name)
    print('[Reading PAMAP2] : DONE')

In [16]:
train_test_files = {'train': config['pamap2']['train_files'],
                    'validation': config['pamap2']['validation_files'],
                    'test': config['pamap2']['test_files']
                    }


read_dataset(train_test_files=train_test_files,
                use_columns=config['pamap2']['feature_columns'],
                output_file_name=os.path.join(config['data_dir']['processed'], config['pamap2']['output_file']))

[Reading PAMAP2] ...
data\processed\pamap2_106.h5
[Reading PAMAP2] : DONE


In [17]:
path = os.path.join(config['data_dir']['processed'], config['pamap2']['output_file'])
f = h5py.File(path, 'r')

x_train = f.get('train').get('inputs')[()]
y_train = f.get('train').get('targets')[()]

x_val = f.get('validation').get('inputs')[()]
y_val = f.get('validation').get('targets')[()]

x_test = f.get('test').get('inputs')[()]
y_test = f.get('test').get('targets')[()]

In [24]:
# replace nan with mean
x_train = np.where(np.isnan(x_train), np.ma.array(x_train, mask=np.isnan(x_train)).mean(axis=0), x_train)
x_val = np.where(np.isnan(x_val), np.ma.array(x_val, mask=np.isnan(x_val)).mean(axis=0), x_val)
x_test = np.where(np.isnan(x_test), np.ma.array(x_test, mask=np.isnan(x_test)).mean(axis=0), x_test)

In [26]:

train_y = tf.keras.utils.to_categorical(y_train, num_classes=19)
test_y = tf.keras.utils.to_categorical(y_test, num_classes=19)
val_y = tf.keras.utils.to_categorical(y_val, num_classes=19)

NameError: name 'tf' is not defined

In [ ]:
import numpy as np
from scipy import stats

def windowz(data, size, use_overlap=True):
    start = 0
    while start < len(data):
        yield start, start + size
        if use_overlap:
            start += (size // 2)
        else:
            start += size


def segment_pa2_test(x_test, y_test, window_size, n_sensor_val):
    segments = np.zeros(((len(x_test) // (window_size)) + 1, window_size, n_sensor_val))
    labels = np.zeros(((len(y_test) // (window_size)) + 1))
    i_segment = 0
    i_label = 0
    for (start, end) in windowz(x_test, window_size, use_overlap=False):
        if end >= x_test.shape[0]:
            pad_len = window_size - len(x_test[start:end])
            segments[i_segment] = x_test[start - pad_len:end]
            m = stats.mode(y_test[start - pad_len:end])
            labels[i_label] = m[0]
        else:
            m = stats.mode(y_test[start:end])
            segments[i_segment] = x_test[start:end]
            labels[i_label] = m[0]
            i_label += 1
            i_segment += 1

    return segments, labels


def segment_pa2(x_train, y_train, window_size, n_sensor_val):
    segments = np.zeros(((len(x_train) // (window_size // 2)) - 1, window_size, n_sensor_val))
    labels = np.zeros(((len(y_train) // (window_size // 2)) - 1))
    i_segment = 0
    i_label = 0
    for (start, end) in windowz(x_train, window_size):
        if len(x_train[start:end]) == window_size:
            m = stats.mode(y_train[start:end])
            segments[i_segment] = x_train[start:end]
            labels[i_label] = m[0]
            i_label += 1
            i_segment += 1
    return segments, labels


def unsegment_pa2_test(y_preds, total_length, window_size):
    unsegmented_preds = np.zeros((total_length,))
    start = 0
    end = window_size
    for element in y_preds:
        if end >= total_length:
            end = total_length
        for i in range(start, end):
            unsegmented_preds[i] = element
        start = end
        end += window_size
        # print(start, end)
    return unsegmented_preds


def segment_window_all(x_train, y_train, window_size, n_sensor_val):
    window_segments = np.zeros((len(x_train), window_size, n_sensor_val))
    labels = np.zeros((len(y_train),))

    total_len = len(x_train)

    for i in range(total_len):
        end = i + window_size

        if end > total_len:
            pad_len = end - total_len
            window_segments[i] = x_train[i - pad_len:end]
            labels[i] = y_train[total_len - 1]
        else:
            window_segments[i] = x_train[i:end]
            labels[i] = y_train[end - 1]

    return window_segments, labels

In [ ]:
import os

import h5py
import numpy as np
import tensorflow as tf
import yaml

# from ._data_reader import read_dataset
# from ._sliding_window import segment_pa2, segment_window_all


def get_pamap2_data(verbose=False):
    config_file = open('configs/data.yaml', mode='r')
    data_config = yaml.load(config_file, Loader=yaml.FullLoader)
    config = data_config['pamap2']

    train_test_files = {'train': config['train_files'],
                        'validation': config['validation_files'],
                        'test': config['test_files']
                        }

    read_dataset(train_test_files=train_test_files,
                 use_columns=config['feature_columns'],
                 output_file_name=os.path.join(data_config['data_dir']['processed'], config['output_file']))

    path = os.path.join(data_config['data_dir']['processed'], config['output_file'])
    f = h5py.File(path, 'r')

    x_train = f.get('train').get('inputs')[()]
    y_train = f.get('train').get('targets')[()]

    x_val = f.get('validation').get('inputs')[()]
    y_val = f.get('validation').get('targets')[()]

    x_test = f.get('test').get('inputs')[()]
    y_test = f.get('test').get('targets')[()]

    if verbose:
        print("x_train shape = ", x_train.shape)
        print("y_train shape =", y_train.shape)

        print("x_val shape = ", x_val.shape)
        print("y_val shape =", y_val.shape)

        print("x_test shape =", x_test.shape)
        print("y_test shape =", y_test.shape)

    x_train = x_train[::3, :]
    y_train = y_train[::3]
    x_val = x_val[::3, :]
    y_val = y_val[::3]
    x_test = x_test[::3, :]
    y_test = y_test[::3]

    if verbose:
        print("x_train shape(downsampled) = ", x_train.shape)
        print("y_train shape(downsampled) =", y_train.shape)
        print("x_val shape(downsampled) = ", x_val.shape)
        print("y_val shape(downsampled) =", y_val.shape)
        print("x_test shape(downsampled) =", x_test.shape)
        print("y_test shape(downsampled) =", y_test.shape)

    # replace nan with mean
    x_train = np.where(np.isnan(x_train), np.ma.array(x_train, mask=np.isnan(x_train)).mean(axis=0), x_train)
    x_val = np.where(np.isnan(x_val), np.ma.array(x_val, mask=np.isnan(x_val)).mean(axis=0), x_val)
    x_test = np.where(np.isnan(x_test), np.ma.array(x_test, mask=np.isnan(x_test)).mean(axis=0), x_test)

    n_sensor_val = len(config['feature_columns']) - 1
    train_x, train_y = segment_pa2(x_train, y_train, config['window_size'], n_sensor_val)
    val_x, val_y = segment_pa2(x_val, y_val, config['window_size'], n_sensor_val)
    test_x, test_y = segment_window_all(x_test, y_test, config['window_size'], n_sensor_val)

    if verbose:
        print("train_x shape =", train_x.shape)
        print("train_y shape =", train_y.shape)
        print('train_y distribution', np.unique(train_y, return_counts=True))

        print("val_x shape =", val_x.shape)
        print("val_y shape =", val_y.shape)
        print('val_y distribution', np.unique(val_y, return_counts=True))

        print("test_x shape =", test_x.shape)
        print("test_y shape =", test_y.shape)
        print('test_y distribution', np.unique(test_y, return_counts=True))

    train_y = tf.keras.utils.to_categorical(train_y, num_classes=19)
    test_y = tf.keras.utils.to_categorical(test_y, num_classes=19)
    val_y = tf.keras.utils.to_categorical(val_y, num_classes=19)

    if verbose:
        print("unique test_y", np.unique(test_y))
        print("unique train_y", np.unique(train_y))
        print("test_y[1]=", test_y[1])

        print("train_y shape(1-hot) =", train_y.shape)
        print("val_y shape(1-hot) =", val_y.shape)
        print("test_y shape(1-hot) =", test_y.shape)

    return (train_x, train_y), (val_x, val_y), (test_x, test_y), y_test